In [1]:
import pandas as pd
import numpy as numpy
import category_encoders as ce

# Armado de sets de entrenamiento

In [2]:
train = pd.DataFrame()

In [3]:
for ventana in range(1,4):
    
    features_pageviews = pd.read_pickle("../data/features/ventana_{}_pageviews.pkl".format(ventana))
    features_conversion = pd.read_pickle("../data/features/ventana_{}_conversion.pkl".format(ventana))
    features_devicedata = pd.read_pickle("../data/features/ventana_{}_devicedata.pkl".format(ventana))
    
    label = pd.read_pickle("../data/labels/labels_trimestre_{}.pkl".format(ventana+1))

    
#    print("VUELTA {}---------------------".format(ventana))
#    print(features_pageviews.columns)
#    print(features_conversion.columns)
#    print(features_devicedata.columns)
#    print(label.columns)
        
    train_actual = label.merge(features_pageviews,left_index = True, right_index = True, how = 'left')
    train_actual = train_actual.merge(features_conversion,left_index = True, right_index = True, how = 'left')
    train_actual = train_actual.merge(features_devicedata,left_index = True, right_index = True, how = 'left')
    
    train = train.append(train_actual)

In [4]:
train.head()

,USER_ID,convirtio,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_top_en_mes_1,page_top_en_mes_2,page_top_en_mes_3,vistas_page_top_en_mes_1,...,busquedas_termino_top_mes_3,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3,convirtio_en_trimestre_actual,cant_vistas_no_md_trimestre,cant_vistas_md_trimestre
0,6574,False,1841.0,710.0,375.0,756.0,10.0,10.0,10.0,140.0,...,756.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,190.0
1,2639,True,623.0,250.0,192.0,181.0,10.0,2.0,10.0,49.0,...,181.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,51.0
2,3898,False,419.0,127.0,78.0,214.0,10.0,2.0,2.0,26.0,...,214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,26.0
3,7996,False,346.0,117.0,118.0,111.0,14.0,85.0,2.0,42.0,...,111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,80.0
4,6111,False,1281.0,311.0,191.0,779.0,10.0,10.0,10.0,106.0,...,779.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.0,3.0


In [5]:
train['convirtio_en_mes_1'].isnull().sum()

34564

### Formateo los datos

In [10]:
train.columns

Index(['USER_ID', 'convirtio', 'cantidad_paginas_vistas_trimestre',
       'cant_paginas_vistas_en_mes_1', 'cant_paginas_vistas_en_mes_2',
       'cant_paginas_vistas_en_mes_3', 'page_top_en_mes_1',
       'page_top_en_mes_2', 'page_top_en_mes_3', 'vistas_page_top_en_mes_1',
       'vistas_page_top_en_mes_2', 'vistas_page_top_en_mes_3',
       'sitio_mas_visitado_en_mes_1', 'sitio_mas_visitado_en_mes_2',
       'sitio_mas_visitado_en_mes_3', 'vistas_site_top_mes_1',
       'vistas_site_top_mes_2', 'vistas_site_top_mes_3',
       'termino_top_en_mes_1', 'termino_top_en_mes_2', 'termino_top_en_mes_3',
       'busquedas_termino_top_mes_1', 'busquedas_termino_top_mes_2',
       'busquedas_termino_top_mes_3', 'convirtio_en_mes_1',
       'convirtio_en_mes_2', 'convirtio_en_mes_3',
       'convirtio_multiple_en_mes_1', 'convirtio_multiple_en_mes_2',
       'convirtio_multiple_en_mes_3', 'convirtio_en_trimestre_actual',
       'cant_vistas_no_md_trimestre', 'cant_vistas_md_trimestre'],
      

#### Columnas categoricas

In [7]:
columnas_categoricas = [   'page_top_en_mes_1', 'page_top_en_mes_2','page_top_en_mes_3',
                        'termino_top_en_mes_1', 'termino_top_en_mes_2', 'termino_top_en_mes_3',
                       'sitio_mas_visitado_en_mes_1','sitio_mas_visitado_en_mes_2', 'sitio_mas_visitado_en_mes_3',]

In [8]:
train[columnas_categoricas] = train[columnas_categoricas].fillna(-1)
train[columnas_categoricas] = train[columnas_categoricas].astype("int16")

#### Columnas con valores numericos

In [9]:
columnas_contar = ['cantidad_paginas_vistas_trimestre','cant_paginas_vistas_en_mes_1', 'cant_paginas_vistas_en_mes_2',
                   'cant_paginas_vistas_en_mes_3','busquedas_termino_top_mes_1', 'busquedas_termino_top_mes_2',
                   'busquedas_termino_top_mes_3','vistas_page_top_en_mes_1', 'vistas_page_top_en_mes_2', 'vistas_page_top_en_mes_3',
                   'vistas_site_top_mes_1', 'vistas_site_top_mes_2', 'vistas_site_top_mes_3','cant_vistas_no_md_trimestre',
                    'cant_vistas_md_trimestre' ]

In [10]:
train[columnas_contar] = train[columnas_contar].fillna(0)
train[columnas_contar] = train[columnas_contar].astype("int64")

#### Columnas booleanas

In [11]:
columnas_booleanas = ['convirtio_en_mes_1', 'convirtio_en_mes_2', 'convirtio_en_mes_3',
       'convirtio_multiple_en_mes_1', 'convirtio_multiple_en_mes_2',
       'convirtio_multiple_en_mes_3',"convirtio_en_trimestre_actual"]

In [12]:
train["convirtio"] = train["convirtio"] * 1

In [13]:
train[columnas_booleanas] = train[columnas_booleanas].fillna(0)
train[columnas_booleanas] = train[columnas_booleanas].astype("int8")

---

In [14]:
train.head(15)

,USER_ID,convirtio,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_top_en_mes_1,page_top_en_mes_2,page_top_en_mes_3,vistas_page_top_en_mes_1,...,busquedas_termino_top_mes_3,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3,convirtio_en_trimestre_actual,cant_vistas_no_md_trimestre,cant_vistas_md_trimestre
0,6574,0,1841.0,710.0,375.0,756.0,10,10,10,140.0,...,756.0,0,0,0,0,0,0,0,0.0,190.0
1,2639,1,623.0,250.0,192.0,181.0,10,2,10,49.0,...,181.0,0,0,0,0,0,0,0,2.0,51.0
2,3898,0,419.0,127.0,78.0,214.0,10,2,2,26.0,...,214.0,0,0,0,0,0,0,0,21.0,26.0
3,7996,0,346.0,117.0,118.0,111.0,14,85,2,42.0,...,111.0,0,0,0,0,0,0,0,0.0,80.0
4,6111,0,1281.0,311.0,191.0,779.0,10,10,10,106.0,...,779.0,0,0,0,0,0,0,0,155.0,3.0
5,2378,0,865.0,281.0,177.0,407.0,85,85,2,69.0,...,407.0,0,0,0,0,0,0,0,0.0,128.0
6,3868,0,288.0,8.0,231.0,49.0,85,168,2,2.0,...,49.0,0,0,0,0,0,0,0,0.0,28.0
7,3861,0,400.0,156.0,118.0,126.0,10,10,10,64.0,...,126.0,0,0,0,0,0,0,0,0.0,63.0
8,2587,0,291.0,131.0,52.0,108.0,10,10,10,34.0,...,108.0,0,0,0,0,0,0,0,0.0,47.0
9,5845,0,386.0,130.0,68.0,188.0,10,10,10,30.0,...,188.0,0,0,0,0,0,0,0,47.0,16.0


In [15]:
train.set_index("USER_ID", inplace = True)

In [16]:
train.to_pickle("../data/train_completo.pkl")

### Armado de set prediccion

In [17]:
    
features_pageviews = pd.read_pickle("../data/features/predecir_pageviews.pkl")
features_conversion = pd.read_pickle("../data/features/predecir_conversion.pkl")
features_devicedata = pd.read_pickle("../data/features/predecir_devicedata.pkl")
    
features_pred = features_pageviews.merge(features_conversion,left_index = True, right_index = True, how = 'left')
features_pred = features_pred.merge(features_devicedata,left_index = True, right_index = True, how = 'left')

In [18]:
features_pred.head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_top_en_mes_1,page_top_en_mes_2,page_top_en_mes_3,vistas_page_top_en_mes_1,vistas_page_top_en_mes_2,vistas_page_top_en_mes_3,...,busquedas_termino_top_mes_3,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3,convirtio_en_trimestre_actual,cant_vistas_no_md_trimestre,cant_vistas_md_trimestre
USER_ID,,,,,,,,,,,,,,,,,,,,,
0,1519,527,490,502,2,2,2,109,88,92,...,502,0,0,0,0,0,0,0,0,112
1,477,138,140,199,2,2,2,30,31,57,...,199,0,0,0,0,0,0,0,9,50
2,651,222,142,287,2,2,21,44,17,45,...,287,0,0,0,0,0,0,0,19,55
3,381,63,121,197,2,2,2,13,21,47,...,197,0,0,0,0,0,0,0,5,52
4,627,284,278,65,27,2,2,52,59,11,...,65,0,0,0,0,0,0,0,46,2


In [19]:
features_pred.to_pickle("../data/prediccion_completo.pkl")

---

## Binary encode

In [20]:
train = pd.read_pickle("../data/train_completo.pkl")

In [21]:
label = train["convirtio"]
train.drop("convirtio",axis = 1, inplace = True )

In [22]:
columnas_categoricas = [   'page_top_en_mes_1', 'page_top_en_mes_2','page_top_en_mes_3',
                        'termino_top_en_mes_1', 'termino_top_en_mes_2', 'termino_top_en_mes_3',
                       'sitio_mas_visitado_en_mes_1','sitio_mas_visitado_en_mes_2', 'sitio_mas_visitado_en_mes_3',]

In [23]:
encoder = ce.BinaryEncoder(cols=columnas_categoricas)
train_binary = encoder.fit_transform(train)



In [24]:

train_binary.columns

Index(['cantidad_paginas_vistas_trimestre', 'cant_paginas_vistas_en_mes_1',
       'cant_paginas_vistas_en_mes_2', 'cant_paginas_vistas_en_mes_3',
       'page_top_en_mes_1_0', 'page_top_en_mes_1_1', 'page_top_en_mes_1_2',
       'page_top_en_mes_1_3', 'page_top_en_mes_1_4', 'page_top_en_mes_1_5',
       'page_top_en_mes_1_6', 'page_top_en_mes_1_7', 'page_top_en_mes_2_0',
       'page_top_en_mes_2_1', 'page_top_en_mes_2_2', 'page_top_en_mes_2_3',
       'page_top_en_mes_2_4', 'page_top_en_mes_2_5', 'page_top_en_mes_2_6',
       'page_top_en_mes_2_7', 'page_top_en_mes_3_0', 'page_top_en_mes_3_1',
       'page_top_en_mes_3_2', 'page_top_en_mes_3_3', 'page_top_en_mes_3_4',
       'page_top_en_mes_3_5', 'page_top_en_mes_3_6', 'page_top_en_mes_3_7',
       'vistas_page_top_en_mes_1', 'vistas_page_top_en_mes_2',
       'vistas_page_top_en_mes_3', 'sitio_mas_visitado_en_mes_1_0',
       'sitio_mas_visitado_en_mes_1_1', 'sitio_mas_visitado_en_mes_1_2',
       'sitio_mas_visitado_en_mes_1_3', '

In [25]:
train_binary["convirtio"] = label

In [26]:
train_binary.head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_top_en_mes_1_0,page_top_en_mes_1_1,page_top_en_mes_1_2,page_top_en_mes_1_3,page_top_en_mes_1_4,page_top_en_mes_1_5,...,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3,convirtio_en_trimestre_actual,cant_vistas_no_md_trimestre,cant_vistas_md_trimestre,convirtio
USER_ID,,,,,,,,,,,,,,,,,,,,,
6574,1841.0,710.0,375.0,756.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,190.0,0
2639,623.0,250.0,192.0,181.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,51.0,1
3898,419.0,127.0,78.0,214.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,21.0,26.0,0
7996,346.0,117.0,118.0,111.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,80.0,0
6111,1281.0,311.0,191.0,779.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,155.0,3.0,0


In [27]:
train_binary.to_pickle("../data/train_codificado.pkl")

In [28]:
pred = pd.read_pickle("../data/prediccion_completo.pkl")

In [29]:
pred_binary = encoder.transform(pred)



In [30]:
pred_binary.to_pickle("../data/prediccion_codificado.pkl")